In [1]:
import torch
import json
import csv
from os import walk
from os import makedirs
from tqdm import tqdm
from transformers import BertJapaneseTokenizer

In [2]:
bert = torch.load('./bert-fmfnb-42.pt')
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")    

In [3]:
TARGET = 42
TYPE = 'dev'
path = f'../tsv/nearest-k-candidates/n-100/{TARGET}/{TYPE}/' ## PATH
savePath = f'./tsv/nearest-k-candidates/n-100/{TARGET}/confidence/'
filenames = next(walk(path), (None, None, []))[2]
makedirs(savePath, exist_ok=True)

In [17]:
from torch.nn import Softmax

sm = Softmax(dim=1)
input = tensor([-1997, 10])
print(input)
print(sm(input))

NameError: name 'tensor' is not defined

In [50]:

def FindConfidence(label, paragraphs, succ, fail, total):
    confidentScores = []
    for paragraph in paragraphs:
        content = paragraph['content']
        tokenized = tokenizer.tokenize(content)
        tokenized.insert(0, '[CLS]')
        tokenized.append('[SEP]')
        tokens = tokenizer.convert_tokens_to_ids(tokenized)
        tensor_tokens = torch.tensor([tokens])
        output = bert(tensor_tokens)
        sm = Softmax(dim=1)
        score = sm(output[0]).cpu().detach().numpy()[0]
        print(score)
        if label == 1:
            confidentScores.append(score[0])
        elif label == 0:
            confidentScores.append(score[1])
        #score = output[0].cpu().detach().numpy()
        #confidentScores.append(score[0][0]-score[0][1])    
    #print(f'Label: {label}')
    #print(f'ConfidentScores: {confidentScores}')
    print(label)
    print(confidentScores)
    print('---')
    singularIndex = confidentScores.index(max(confidentScores))
    singularValue = max(confidentScores)
    if singularValue > 0.5:
        succ += 1
    else:
        fail += 1
    total += 1
    """
    if label == 1:
        singularIndex = confidentScores.index(max(confidentScores))
        singularValue = max(confidentScores)
        #print(f"Got maximum score at INDEX {singularIndex}, CONFIDENCE {singularValue}:\n\tSCORE:{paragraphs[singularIndex]['score']}\n\tCONTENT:{paragraphs[singularIndex]['content'].replace(' ', '')[:64]}...")
        if singularValue > 0:
            succ += 1
        else:
            fail += 1
        total += 1
    else:
        singularIndex = confidentScores.index(min(confidentScores))
        singularValue = min(confidentScores)
        #print(f"Got minimum score at INDEX {singularIndex}, CONFIDENCE {singularValue}:\n\tSCORE:{paragraphs[singularIndex]['score']}\n\tCONTENT:{paragraphs[singularIndex]['content'].replace(' ', '')[:64]}...")
        if singularValue < 0:
            succ += 1
        else:
            fail += 1
        total += 1
    #print('-'*128)
    """
    status = {'label': label, 'confidence_scores': str(confidentScores), 'index': singularIndex, 'confidence': str(singularValue), 'score': paragraphs[singularIndex]['score'], 'content': paragraphs[singularIndex]['content'].replace(' ', '')}
    return succ, fail, total, status

In [51]:
totalNum = 0
successNum = 0
failedNum = 0
result = []
for file in filenames[:5]:
    with open(f'{path}/{file}', 'r', encoding='utf-8') as f:
        data = json.load(f)
    if data['label'] == '10':
        actr_label = 1
    else:
        actr_label = 0
    paragraphs = data['status']
    successNum, failedNum, totalNum, status = FindConfidence(actr_label, paragraphs, successNum, failedNum, totalNum)
    result.append(status)

[0.02311138 0.9768886 ]
[0.00364553 0.99635446]
[0.00988815 0.9901119 ]
[0.00321145 0.9967886 ]
0
[0.9768886, 0.99635446, 0.9901119, 0.9967886]
---
[0.99880147 0.00119855]
[0.90832025 0.09167976]
[0.2851887 0.7148113]
[0.01427708 0.98572296]
1
[0.99880147, 0.90832025, 0.2851887, 0.014277075]
---
[9.9981302e-01 1.8700887e-04]
[0.00189862 0.9981014 ]
[0.00949643 0.99050355]
1
[0.999813, 0.0018986181, 0.009496427]
---
[9.9937338e-01 6.2669005e-04]
[0.00184461 0.9981554 ]
[0.93988174 0.06011825]
[5.894871e-04 9.994105e-01]
1
[0.9993734, 0.0018446083, 0.93988174, 0.0005894871]
---
[0.9976236  0.00237638]
[0.79585034 0.20414963]
[0.9901904  0.00980958]
[0.02728594 0.972714  ]
[0.00203313 0.9979668 ]
0
[0.0023763776, 0.20414963, 0.009809576, 0.972714, 0.9979668]
---


In [52]:
print(result)

[{'label': 0, 'confidence_scores': '[0.9768886, 0.99635446, 0.9901119, 0.9967886]', 'index': 3, 'confidence': '0.9967886', 'score': 1, 'content': '魔物や外国の軍隊から町を守らなきゃいけない）しょせん、ディアディンは農夫の息子だ兵士になりたいと言ったって、家族をすてて家を出ていくことはできないだろういつかは自分も農夫になるのだということは、心の底ではわかっていた本物の騎士になれるのは、泣き虫のリックのほうで、自分じゃないでも、それでも、まだ、そのころは夢を見ていられた将来は未定で、今日と同じ幸福な毎日が続いていくことだけは変わらないと思っていた予測があんな形で裏切られることになるとは、思いもしなかった数年後、ディアディンは願いどおり、兵士になった大人になるのを待たずしてきっかけは、あの日——ちょっとした偶然がかさなって始まった、不幸の連鎖ディアディンが、ほんの数分早く、あの場にかけつけていれば……あの日にかぎって、塾の先生に質問して、残ったりしなければよかった塾の帰り道、友達と冗談なんて言いあわなければよかったそれとも、それとも……ほかに、もっと、しようがあったはずなのにディアディンが丘の上まで辿りついたとき、セオドリックはミュルトと二人で、一本杉にのぼっていたいつもは怖がって、絶対にのぼろうとしなかったリックなぜ、あの日にかぎって、そんなことをしたのか'}, {'label': 1, 'confidence_scores': '[0.99880147, 0.90832025, 0.2851887, 0.014277075]', 'index': 0, 'confidence': '0.99880147', 'score': 9, 'content': '戦闘ができる霊能者たちはね家系や持つ力次第だけど、私はこの刀と守り刀を清めて、力を持ってしまった霊の怨霊を切り払ってるのよほー刀でなければいけないのか昔から、鉄と刃というのは魔を祓う力があると言われているのよそれを、神酒で清めて、祭壇で祀って、力を込めるの俺でもできんの無理じゃないこういう力を込めるというか、浄化する力もまちまちだし……そういういざって時はどうすればいいん

In [53]:
print(successNum, failedNum, totalNum)
print(f'Ideal Accuracy = {(successNum/totalNum):.4%}')
writeTarget = {
    'ideal_accuracy': f'{(successNum/totalNum):.4%}',
    'work_list': result
}
with open(f'{savePath}/{TYPE}_result.json', 'w+', encoding='utf-8') as f:
    json.dump(writeTarget, f)

5 0 5
Ideal Accuracy = 100.0000%


In [ ]:
"""
totalWorks = 0
successWorks = 0
failedWorks = 0

confidentScores = []
for paragraph in paragraphs:
    content = paragraph['content']
    tokenized = tokenizer.tokenize(content)
    tokenized.insert(0, '[CLS]')
    tokenized.append('[SEP]')
    tokens = tokenizer.convert_tokens_to_ids(tokenized)
    tensor_tokens = torch.tensor([tokens])
    output = bert(tensor_tokens)
    #print(output[0])
    score = output[0].cpu().detach().numpy()
    confidentScores.append(score[0][0]-score[0][1])
print(f'ConfidentScores: {confidentScores}')
if actr_label == 1:
    singularIndex = confidentScores.index(max(confidentScores))
    singularValue = max(confidentScores)
    print(f"Got maximum score at INDEX {singularIndex}, CONFIDENCE {singularValue}:\n\tSCORE:{paragraphs[singularIndex]['score']}\n\tCONTENT:{paragraphs[singularIndex]['content'].replace(' ', '')[:64]}...")
    if singularValue > 0:
        successWorks += 1
    else:
        failedWorks += 1
    totalWorks += 1
else:
    singularIndex = confidentScores.index(min(confidentScores))
    singularValue = min(confidentScores)
    print(f"Got minimum score at INDEX {singularIndex}, CONFIDENCE {singularValue}:\n\tSCORE:{paragraphs[singularIndex]['score']}\n\tCONTENT:{paragraphs[singularIndex]['content'].replace(' ', '')[:64]}...")
    if singularValue < 0:
        successWorks += 1
    else:
        failedWorks += 1
    totalWorks += 1
"""

In [32]:
for work in works[15:20]:
    label = work['label']
    content = work['content']
    tokenized = tokenizer.tokenize(content)
    tokenized.insert(0, '[CLS]')
    #print(tokenized)
    tokens = tokenizer.convert_tokens_to_ids(tokenized)
    tensor_tokens = torch.tensor([tokens])
    output = bert(tensor_tokens)
    print(label)
    print(output[0])

1
tensor([[ 2.6664, -2.7127]], device='cuda:0', grad_fn=<GatherBackward>)
1
tensor([[ 3.9455, -4.1770]], device='cuda:0', grad_fn=<GatherBackward>)
0
tensor([[-2.6080,  3.0158]], device='cuda:0', grad_fn=<GatherBackward>)
0
tensor([[-2.9660,  3.2850]], device='cuda:0', grad_fn=<GatherBackward>)
0
tensor([[-0.5614,  1.0273]], device='cuda:0', grad_fn=<GatherBackward>)


In [5]:
import workFactory

works = workFactory.load_tsv_dataset('./n-100-1st-test.tsv')

In [6]:
print(works[0])

{'label': 0, 'content': '力 を 、 西側 に 要請 する ため の な 言い ながら アロイス は ちらり と ヴォルフ へ 視線 を 流し た 彼 の 言う 『 西側 』 に は 、 当然 ブラッド ロー も 含ま れ て いる の だろう ヴォルフ が 僅か に 息 を 飲ん だ 知っ て の 通り 、 僕 ら は オプシルナーヤ へ の 反乱 計画 を 立て て て 、 それ に 向け て 密か に 準備 し とる ところ や や けど 当然 の こと ながら 、 圧倒 的 に 戦力 が 足り や ん もっと 強大 な 戦力 が 欲しい ところ や と は いえ 西側 に 戦力 を 頼る に も 、 交渉 に 足る 十分 な 条件 が 何 一 つ 無い 状況 やっ た 向こう に し ちゃ 敵領 に わざわざ 軍 を 送り込む わけ や し 、 と アロイス は 肩 を 竦め た や けど ホフマン の 爆弾 は 別 や 街 を 一気 に 吹き飛ばす 破壊 兵器 ―― 噂 が ほんま か 分から ん けど 、 そんな もの を オプシルナーヤ が うようよ し て いる 東 ヘルデンズ の 人間 が 拾っ たって 知っ たら 、 流石 に 西側 の 反応 も 変わる 僕 ら は それ を 利用 し て やり たい ん や 言い ながら 、 段々 と アロイス の 言葉 に は 熱 が 籠もっ て いっ た 固い 決意 の 込め られ た それ は 、 同時 に 切実 な 響き を 帯び て いる それ に 水 を 差す よう に 、 ヴォルフ が だ が … … と 言葉 を 続ける 仮 に ホフマン の 爆弾 が あっ た と し て も 、 今 の お前 ら で は 、 結局 の 軍事 力 を 動かす に 至ら ない ん じゃ ない の か ここ に いる ヘルデンズ 人 の 今 の 有様 も 正直 不利 だ そう か も な 兄 さん'}


In [8]:
work = works[0]
label = work['label']
content = work['content']

In [9]:
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")    

In [15]:
tokenized = tokenizer.tokenize(content)
print(tokenized)
tokenized.insert(0, '[CLS]')
tokens = tokenizer.convert_tokens_to_ids(tokenized)
print(tokens)

['人間', 'と', 'いう', '括', '##り', 'が', '同じ', '――', 'と', 'いう', 'わけ', 'で', 'は', 'ない', 'ホロ', '##ウ', '##メア', 'で', 'は', '人間', '以外', 'に', 'も', '、', '龍', 'と', '人間', 'の', 'ハーフ', 'の', '龍', '##人', '、', '獣', 'と', '人間', 'の', 'ハーフ', 'の', '獣', '人', '、', '烏', '天狗', 'や', '鬼', 'など', 'の', '妖怪', '、', '人', '##語', 'を', '解', '##する', '炎', 'を', '纏', '##っ', 'た', '鼠', 'など', 'など', '、', '様々', 'な', '種族', 'が', '入り', '##乱', '##れ', 'て', '生活', 'し', 'て', 'いる', 'だ', 'が', 'しかし', '、', '異', '種族', '同士', 'が', '同じ', '領土', '内', 'に', '生息', 'し', 'て', 'いる', 'と', '言え', 'ど', '、', 'い', '##がみ', '##合っ', 'て', 'は', 'い', 'ない', '、', 'い', '##がみ', '##合う', '余裕', 'も', 'ない', 'ほど', '、', 'ホロ', '##ウ', 'メ', '##ア', 'は', 'いま', '重大', 'な', '危機', 'に', '直面', 'し', 'て', 'いる', 'の', 'だ', '――', '結晶', '病', 'ダン', 'が', '発し', 'た', '耳', '##慣れ', 'ない', '単語', 'に', '、', 'ペ', 'ポ', 'は', 'オウム', '返し', 'を', 'する', 'ホロ', '##ウ', '##メア', 'でし', 'て', 'いる', '奇', '##病', 'です', 'り', 'に', '言え', 'ば', '、', '体', 'が', '結晶', '化', 'し', 'て', '最終', '的', 'に', 'に', 'なる', '死', 'の', '病', 'です', 'ね',

In [18]:
tensor_tokens = torch.tensor([tokens])
print(tensor_tokens)

tensor([[    2,  1410,    13,   625,  9697, 28477,    14,   552, 18454,    13,
           625,  3133,    12,     9,    80, 14323, 28549, 17165,    12,     9,
          1410,  1000,     7,    28,     6,  2423,    13,  1410,     5,  9241,
             5,  2423, 28514,     6,  5777,    13,  1410,     5,  9241,     5,
          5777,    53,     6,  9324, 18463,    49,  4262,    64,     5,  9954,
             6,    53, 28703,    11,   432,    30,  3695,    11, 13621, 28468,
            10, 23820,    64,    64,     6,  1400,    18, 12945,    14,  1207,
         29349, 28461,    16,  1326,    15,    16,    33,    75,    14,   373,
             6,   532, 12945,  4128,    14,   552,  5295,   186,     7,  3905,
            15,    16,    33,    13,  5498,   614,     6,    21, 19222,  6527,
            16,     9,    21,    80,     6,    21, 19222,  7393, 12614,    28,
            80,  1101,     6, 14323, 28549,   168, 28483,     9,  6739,  9430,
            18,  3989,     7, 13183,    15,    16,  

In [28]:
bert.eval()
output = bert(tensor_tokens)
print(output)

SequenceClassifierOutput(loss=<generator object gather.<locals>.gather_map.<locals>.<genexpr> at 0x7f545de23dd0>, logits=tensor([[-1.6233,  2.0551]], device='cuda:0', grad_fn=<GatherBackward>), hidden_states=None, attentions=None)


In [29]:
print(output[0])

tensor([[-1.6233,  2.0551]], device='cuda:0', grad_fn=<GatherBackward>)


In [7]:
import json
import pandas as pd
data = []
TGS=[ 1, 2, 3, 4, 5, 20, 39, 40, 42, 69, 70, 71, 73, 74, 75, 77, 79, 80, 81, 83, 84, 87, 90, 96, 120, 121, 122, 126, 128, 199, 200, 203, 204, 214, 259, 260, 281, 284, 291]
for TARGET in TGS:
    path = f'../tsv/nearest-k-candidates-offset-faster/n-100/o-256/{TARGET}/confidence' ## PATH
    with open(f'{path}/train_result.json', 'r', encoding='utf-8') as f:
        train = json.load(f)['ideal_accuracy']
    with open(f'{path}/dev_result.json', 'r', encoding='utf-8') as f:
        dev = json.load(f)['ideal_accuracy']
    with open(f'{path}/test_result.json', 'r', encoding='utf-8') as f:
        test = json.load(f)['ideal_accuracy']
    data.append([TARGET, train, dev, test])
print(data)

[[1, '92.0498%', '91.7305%', '92.3430%'], [2, '95.5845%', '96.6387%', '94.9580%'], [3, '99.3553%', '95.6522%', '95.1220%'], [4, '95.0541%', '97.1922%', '95.0324%'], [5, '94.4865%', '93.6027%', '93.6027%'], [20, '94.8613%', '96.3816%', '95.4023%'], [39, '94.0347%', '92.9385%', '93.5080%'], [40, '96.0314%', '94.6494%', '95.0185%'], [42, '99.6288%', '97.2973%', '95.1482%'], [69, '88.7934%', '87.3603%', '88.0584%'], [70, '94.1263%', '93.0282%', '93.6170%'], [71, '94.2159%', '94.3253%', '94.4584%'], [73, '92.2850%', '91.5683%', '91.9056%'], [74, '93.3735%', '92.7711%', '92.3695%'], [75, '89.8271%', '89.6341%', '90.6504%'], [77, '93.7998%', '93.1442%', '92.2170%'], [79, '87.5234%', '84.0399%', '84.7880%'], [80, '97.1138%', '95.0131%', '93.4555%'], [81, '94.5197%', '95.3202%', '95.3202%'], [83, '94.9264%', '94.1176%', '95.0980%'], [84, '94.1782%', '91.3978%', '95.0000%'], [87, '95.6039%', '94.0741%', '95.2030%'], [90, '98.6081%', '94.0171%', '92.7350%'], [96, '98.4062%', '97.1193%', '96.3115%

In [8]:
pd.DataFrame(data)

,0,1,2,3
0,1,92.0498%,91.7305%,92.3430%
1,2,95.5845%,96.6387%,94.9580%
2,3,99.3553%,95.6522%,95.1220%
3,4,95.0541%,97.1922%,95.0324%
4,5,94.4865%,93.6027%,93.6027%
5,20,94.8613%,96.3816%,95.4023%
6,39,94.0347%,92.9385%,93.5080%
7,40,96.0314%,94.6494%,95.0185%
8,42,99.6288%,97.2973%,95.1482%
9,69,88.7934%,87.3603%,88.0584%


In [9]:
TARGET = 3
path = f'../tsv/nearest-k-candidates/n-100/{TARGET}/confidence' ## PATH
with open(f'{path}/train_result.json', 'r', encoding='utf-8') as f:
    train = json.load(f)['ideal_accuracy']
with open(f'{path}/dev_result.json', 'r', encoding='utf-8') as f:
    dev = json.load(f)['ideal_accuracy']
with open(f'{path}/test_result.json', 'r', encoding='utf-8') as f:
    test = json.load(f)['ideal_accuracy']
print(train, dev, test)

FileNotFoundError: [Errno 2] No such file or directory: '../tsv/nearest-k-candidates/n-100/3/confidence/train_result.json'

In [22]:
def getAvgCount(path):
    T10 = []
    Tu10 = []
    F10 = []
    Fu10 = []
    files = next(walk(path), (None, None, []))[2]
    for t in tqdm(files):
        with open(f'{path}/{t}', 'r') as f:
            data = json.load(f)
        label = data['label']
        count = data['match_num']
        if label == '10':
            if count > 10:
                T10.append(count)
            else:
                Tu10.append(count)
        elif label == '01':
            if count > 10:
                F10.append(count)
            else:
                Fu10.append(count)
    TAVG10 = sum(T10)/len(T10)
    TAVGu10 = sum(Tu10)/len(Tu10)
    FAVG10 = sum(F10)/len(F10)
    FAVGu10 = sum(Fu10)/len(Fu10)
    return TAVG10, len(T10), TAVGu10, len(Tu10), FAVG10, len(F10),FAVGu10, len(Fu10)

In [24]:
import json
from tqdm import tqdm
from os import walk
import pandas as pd
trainData = []
devData = []
testData = []

TGS=[ 1, 2, 3, 4, 5, 20, 39, 40, 42, 69, 70, 71, 73, 74, 75, 77, 79, 80, 81, 83, 84, 87, 90, 96, 120, 121, 122, 126, 128, 199, 200, 203, 204, 214, 259, 260, 281, 284, 291]
for TARGET in TGS:
    tavg10, ltavg10, tavgu10, ltavgu10, favg10, lfavg10, favgu10, lfavgu10 =  getAvgCount(f'../tsv/nearest-k-candidates-offset-faster/n-100/o-256/{TARGET}/train')
    trainData.append([TARGET, tavg10, ltavg10, tavgu10, ltavgu10, favg10, lfavg10, favgu10, lfavgu10])
    
    tavg10, ltavg10, tavgu10, ltavgu10, favg10, lfavg10, favgu10, lfavgu10 =  getAvgCount(f'../tsv/nearest-k-candidates-offset-faster/n-100/o-256/{TARGET}/devs')
    devData.append([TARGET, tavg10, ltavg10, tavgu10, ltavgu10, favg10, lfavg10, favgu10, lfavgu10])
    
    tavg10, ltavg10, tavgu10, ltavgu10, favg10, lfavg10, favgu10, lfavgu10 =  getAvgCount(f'../tsv/nearest-k-candidates-offset-faster/n-100/o-256/{TARGET}/test')
    testData.append([TARGET, tavg10, ltavg10, tavgu10, ltavgu10, favg10, lfavg10, favgu10, lfavgu10])
    
"""
    path = f'../tsv/nearest-k-candidates-offset-faster/n-100/o-256/{TARGET}/' ## PATH
    with open(f'{path}/train_result.json', 'r', encoding='utf-8') as f:
        train = json.load(f)['ideal_accuracy']
    with open(f'{path}/dev_result.json', 'r', encoding='utf-8') as f:
        dev = json.load(f)['ideal_accuracy']
    with open(f'{path}/test_result.json', 'r', encoding='utf-8') as f:
        test = json.load(f)['ideal_accuracy']
    data.append([TARGET, train, dev, test])
 
print(data)
"""

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1901/1901 [00:07<00:00, 242.31it/s]


"\n    path = f'../tsv/nearest-k-candidates-offset-faster/n-100/o-256/{TARGET}/' ## PATH\n    with open(f'{path}/train_result.json', 'r', encoding='utf-8') as f:\n        train = json.load(f)['ideal_accuracy']\n    with open(f'{path}/dev_result.json', 'r', encoding='utf-8') as f:\n        dev = json.load(f)['ideal_accuracy']\n    with open(f'{path}/test_result.json', 'r', encoding='utf-8') as f:\n        test = json.load(f)['ideal_accuracy']\n    data.append([TARGET, train, dev, test])\n \nprint(data)\n"

In [27]:
pd.DataFrame(testData)

,0,1,2,3,4,5,6,7,8
0,1,453.676101,318,6.500000,22,325.545455,209,5.028846,104
1,2,543.411765,272,6.565217,23,280.422535,213,4.540230,87
2,3,319.834320,169,5.384615,13,151.819277,83,2.144231,104
3,4,317.203791,211,4.384615,13,157.485507,138,3.029703,101
4,5,676.632911,158,4.500000,4,221.183908,87,2.187500,48
5,20,205.047809,251,6.222222,36,173.168478,184,3.152174,138
6,39,461.180049,411,5.228571,35,218.786517,178,1.712598,254
7,40,221.037528,453,5.155172,58,127.310345,261,2.637821,312
8,42,397.267045,176,5.500000,18,271.943182,88,2.359551,89
9,69,278.318777,458,3.676471,136,136.824219,256,2.557325,314
